# Creating WCM File Component using Python
First specify the libraries needed.  In this example the Requests library was used to communicate with the Portal/WCM server.  
Note: This example uses [Python f-strings](https://realpython.com/python-f-strings/) which were introduced in v3.6.

In [1]:
import requests
import xml.etree.ElementTree as ET

You'll need the URL for the WCM REST API.  I split this into two variables, and you'll see why a little later on.  Replace the `hostname:port` with your values in the `baseURL` variable.

In [2]:
baseURL = 'http://portal9.home.parrill.net:10039'
restURL = f'{baseURL}/wps/mycontenthandler/wcmrest'

Several of the WCM REST API calls are available in XML and JSON.  This example uses XML.

In [3]:
# Define the XML namespaces
ns = {'atom': 'http://www.w3.org/2005/Atom',
      'wcm': 'http://www.ibm.com/xmlns/wcm/8.0'}

The first step is to get the UUID of the library which will contain the file component.  While this can be done using the user interface, it's more fun to do it programmatically.  In this example we'll search by the library title.  
A library named `Test Upload` had been previously created.

In [4]:
# Initialize our Requests session parameters.  You'll want to replace the credentials with ones for your own environment
s = requests.Session()
s.auth=('wpsadmin', 'password')  # Normally you wouldn't hard-code something like this!

# Use the Query function to find the title we're looking for.  If you know the exact name you can use
# the 'title' query parameter, or you can use 'titlelike' if you have a partial name.
# If mulitple matches are found this example is only using the first match
libName = 'Test Upload'
r = s.get(f'{restURL}/query?type=Library&titlelike={libName}')
root = ET.fromstring(r.text)
id = root.find('atom:entry/atom:id', ns).text
libID = id.split(':')[1]

print(r.text)
print(f'The library ID is: {libID}')

<?xml version="1.0" encoding="UTF-8"?><feed xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="http://www.ibm.com/xmlns/wcm/8.0">
    <id>wcmrest:query?titlelike=Test%20Upload<![CDATA[&type=Library]]></id>
    <title>wcmrest:query?titlelike=Test%20Upload<![CDATA[&type=Library]]></title>
    <updated>2018-11-09T22:53:19.991Z</updated>
    <entry>
        <id>wcmrest:38606f98-ffb1-4b8e-bd81-3a1692b748fe</id>
        <title>Test Upload</title>
        <summary/>
        <wcm:type>Library</wcm:type>
        <updated>2018-11-09T18:47:04.903Z</updated>
        <link rel="edit" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Edit"/>
        <link rel="alternate" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Read"/>
        <link rel="library" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Li

In [5]:
r.text

'<?xml version="1.0" encoding="UTF-8"?><feed xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="http://www.ibm.com/xmlns/wcm/8.0">\n    <id>wcmrest:query?titlelike=Test%20Upload<![CDATA[&type=Library]]></id>\n    <title>wcmrest:query?titlelike=Test%20Upload<![CDATA[&type=Library]]></title>\n    <updated>2018-11-09T22:53:19.991Z</updated>\n    <entry>\n        <id>wcmrest:38606f98-ffb1-4b8e-bd81-3a1692b748fe</id>\n        <title>Test Upload</title>\n        <summary/>\n        <wcm:type>Library</wcm:type>\n        <updated>2018-11-09T18:47:04.903Z</updated>\n        <link rel="edit" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Edit"/>\n        <link rel="alternate" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Read"/>\n        <link rel="library" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmF

In addition to the library ID, the library link will also be required.  This could be parsed out of the info above, but in this example we're making a call to the library object to get additional information about it.  
The `<link rel="self" ... >` tag is the one to be parsed for the URL.

In [6]:
r = s.get(f'{restURL}/Library/{libID}')
root = ET.fromstring(r.text)
libLnk = root.find('atom:link/[@rel="self"]', ns).attrib['href']

print(r.text)
print(f'The library link is: {libLnk}')

<?xml version="1.0" encoding="UTF-8"?><entry xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="http://www.ibm.com/xmlns/wcm/8.0">
    <id>wcmrest:38606f98-ffb1-4b8e-bd81-3a1692b748fe</id>
    <title>Test Upload</title>
    <summary/>
    <wcm:name>test upload</wcm:name>
    <wcm:type>Library</wcm:type>
    <updated>2018-11-09T18:47:04.903Z</updated>
    <link rel="self" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Read"/>
    <link rel="edit" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Edit"/>
    <link rel="delete" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/wcmrest/Library/38606f98-ffb1-4b8e-bd81-3a1692b748fe" xml:lang="en" label="Delete"/>
    <link rel="access-control" href="/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/ac/access:oid:Z6QReDeJ1EC3OO66BEGJMOC6JO2JMG623E

Notice the library link is a relative path starting with `/wps/mycontenthandler/...`

Next, create the XML which will be sent with the request to create the file component.  This example just creates a raw string with the desired inputs.  Be sure to specify the `href=` parameter using the link to your library.  Adjust the `title`, `name`, and `description` fields as needed.

In [7]:
fileXML = f'<entry xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="wcm/namespace"> \
        <title>Programmatic Test File Component 1</title> \
        <link rel="library" href="{libLnk}"/> \
        <wcm:name>ProgTestFileComponent1</wcm:name> \
        <wcm:description>This is test file component uploaded from Python</wcm:description> \
    </entry>'

Now call the POST interface to create the new file component.  Note that the file is not yet included.  Be sure to save the UUID of the new file component since that will be needed shortly.

In [8]:
r = s.post(f'{restURL}/LibraryFileComponent',
                data = fileXML,
                headers={'Content-Type':'application/atom+xml'})
root = ET.fromstring(r.text)
id = root.find('atom:id', ns).text
fileID = id.split(':')[1]

print(r.text)
print(f'The file ID is: {fileID}')

<?xml version="1.0" encoding="UTF-8"?><entry xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="http://www.ibm.com/xmlns/wcm/8.0">
    <id>wcmrest:c03f61f4-eaaa-4dc7-830e-94f6e5f11a6e</id>
    <title xml:lang="en">Programmatic Test File Component 1</title>
    <summary xml:lang="en"></summary>
    <wcm:name>ProgTestFileComponent1</wcm:name>
    <wcm:type>LibraryFileComponent</wcm:type>
    <updated>2018-11-09T22:53:20.174Z</updated>
    <wcm:created>2018-11-09T22:53:20.174Z</wcm:created>
    <author>
        <wcm:distinguishedName>uid=wpsadmin,o=defaultWIMFileBasedRealm</wcm:distinguishedName>
        <uri>/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/Z9eAeG1CA6R8643D4JM07GPD8MMG663PAJM4C43D4JM4CM9P43O86OPO83O4643</uri>
        <name>wpsadmin</name>
    </author>
    <wcm:owner>
        <wcm:distinguishedName>uid=wpsadmin,o=defaultWIMFileBasedRealm</wcm:distinguishedName>
        <uri>/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/

If successful the file may now be added to the component using the PUT command.  The file is added as binary data, and be sure to specify the file's UUID in the command.
This example uses a local file named `TEST.PDF`.

In [9]:
with open('TEST.PDF', mode='rb') as fh:
    mydata = fh.read()
r = s.put(f'{restURL}/LibraryFileComponent/{fileID}',
          data = mydata,
          headers={'Content-Type':'application/pdf'})

print(r.text)

<?xml version="1.0" encoding="UTF-8"?><entry xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="http://www.ibm.com/xmlns/wcm/8.0">
    <id>wcmrest:c03f61f4-eaaa-4dc7-830e-94f6e5f11a6e</id>
    <title xml:lang="en">Programmatic Test File Component 1</title>
    <summary xml:lang="en"></summary>
    <wcm:name>ProgTestFileComponent1</wcm:name>
    <wcm:type>LibraryFileComponent</wcm:type>
    <updated>2018-11-09T22:53:20.664Z</updated>
    <wcm:created>2018-11-09T22:53:20.174Z</wcm:created>
    <author>
        <wcm:distinguishedName>uid=wpsadmin,o=defaultWIMFileBasedRealm</wcm:distinguishedName>
        <uri>/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/Z9eAeG1CA6R8643D4JM07GPD8MMG663PAJM4C43D4JM4CM9P43O86OPO83O4643</uri>
        <name>wpsadmin</name>
    </author>
    <wcm:owner>
        <wcm:distinguishedName>uid=wpsadmin,o=defaultWIMFileBasedRealm</wcm:distinguishedName>
        <uri>/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/

Verify the item was created and the file is attached by going into the WCM authoring portlet and looking at the Components folder inside the library:  
![alt text](Library_Item_Created.jpg)  
View the details for the new file component:  
![alt text](Library_Item_Details.jpg) 

## Retrieve the item
The new file component can also be easily retrieved using the REST API as well.  
The [WCM Query service](https://www.ibm.com/support/knowledgecenter/SSHRKX_8.5.0/mp/wcm/wcm_rest_adhoc.html) 
can be used to find the UUID of the item using a variety of search parameters.  In this case the UUID is already known from the file component created earlier.  
A WCM component can be read by sending a GET request to the following URI: `/<library-component-api-type>/<itemuuid>`  
The file component uses a type of `LibraryFileComponent`.

In [10]:
r = s.get(f'{restURL}/LibraryFileComponent/{fileID}')

print(r.text)

<?xml version="1.0" encoding="UTF-8"?><entry xmlns="http://www.w3.org/2005/Atom" xmlns:wcm="http://www.ibm.com/xmlns/wcm/8.0">
    <id>wcmrest:c03f61f4-eaaa-4dc7-830e-94f6e5f11a6e</id>
    <title xml:lang="en">Programmatic Test File Component 1</title>
    <summary xml:lang="en"></summary>
    <wcm:name>ProgTestFileComponent1</wcm:name>
    <wcm:type>LibraryFileComponent</wcm:type>
    <updated>2018-11-09T22:53:20.664Z</updated>
    <wcm:created>2018-11-09T22:53:20.174Z</wcm:created>
    <author>
        <wcm:distinguishedName>uid=wpsadmin,o=defaultWIMFileBasedRealm</wcm:distinguishedName>
        <uri>/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/Z9eAeG1CA6R8643D4JM07GPD8MMG663PAJM4C43D4JM4CM9P43O86OPO83O4643</uri>
        <name>wpsadmin</name>
    </author>
    <wcm:owner>
        <wcm:distinguishedName>uid=wpsadmin,o=defaultWIMFileBasedRealm</wcm:distinguishedName>
        <uri>/wps/mycontenthandler/!ut/p/digest!O-zK3jJTx-vVze4JgBmFiw/um/users/profiles/

Parse the data to get the URL for the actual file attachment.

In [11]:
root = ET.fromstring(r.text)
lnk = root.find('atom:content/wcm:resourceUri', ns).text

print(f'The link to the file is: {lnk}')

The link to the file is: /wps/wcm/myconnect/db9ffca9-2e45-4b8a-8106-f17516e7ebc8/1b23dfb5-6beb-4fb9-aa96-be237332d8a8.pdf?MOD=AJPERES&CVID=mrWHwmo


In [12]:
print(f'The full url is: {baseURL}{lnk}')

The full url is: http://portal9.home.parrill.net:10039/wps/wcm/myconnect/db9ffca9-2e45-4b8a-8106-f17516e7ebc8/1b23dfb5-6beb-4fb9-aa96-be237332d8a8.pdf?MOD=AJPERES&CVID=mrWHwmo
